# Neural Sparse 모델 성능 향상 연구: 동의어 강화

## 문제 분석

### 현재 모델의 한계:
1. **Vocabulary Mismatch**: 쿼리 토큰과 문서 토큰 불일치
   - 예: "OpenSearch" (쿼리) ≠ "검색", "개방" (문서)
2. **동의어 처리 부족**: 의미는 같지만 표현이 다른 경우 검색 실패
3. **한국어 특성**: 다양한 표현 방식 (한자어, 외래어, 순우리말)

## 해결 방법 연구

### 1. Knowledge Distillation from Dense Model (추천 ⭐)
### 2. Synonym-Aware IDF Enhancement
### 3. Classification Model for Token Similarity
### 4. Query/Document Expansion

## 1. 환경 설정

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import (
    AutoTokenizer, 
    AutoModel,
    AutoModelForMaskedLM,
    AutoConfig
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 기존 모델 로드 및 문제 재현

In [ ]:
MODEL_DIR = "./opensearch-korean-neural-sparse-v1"

# Tokenizer 및 IDF 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

with open(os.path.join(MODEL_DIR, "idf.json"), 'r', encoding='utf-8') as f:
    idf_dict = json.load(f)

print(f"✓ Tokenizer 로드: vocab_size={tokenizer.vocab_size:,}")
print(f"✓ IDF 딕셔너리: {len(idf_dict):,}개 토큰")

### 2.1. 문제 재현: Vocabulary Mismatch 분석

In [ ]:
# 문제가 되는 쿼리-문서 쌍
problematic_cases = [
    {
        "query": "OpenSearch 벡터 검색",
        "document": "OpenSearch는 강력한 검색 및 분석 엔진입니다.",
        "expected_match": ["OpenSearch", "검색"],
    },
    {
        "query": "인공지능 기술",
        "document": "AI 기반 시스템이 발전하고 있습니다.",
        "expected_match": [("인공지능", "AI"), ("기술", "시스템")],
    },
    {
        "query": "딥러닝 학습",
        "document": "심층 신경망 훈련 방법론",
        "expected_match": [("딥러닝", "심층 신경망"), ("학습", "훈련")],
    },
]

print("="*80)
print("문제 분석: Vocabulary Mismatch")
print("="*80)

for case in problematic_cases:
    query = case['query']
    doc = case['document']
    
    query_tokens = tokenizer.tokenize(query)
    doc_tokens = tokenizer.tokenize(doc)
    
    overlap = set(query_tokens) & set(doc_tokens)
    
    print(f"\n쿼리: {query}")
    print(f"  토큰: {query_tokens}")
    print(f"\n문서: {doc}")
    print(f"  토큰: {doc_tokens}")
    print(f"\n겹치는 토큰: {overlap if overlap else '없음 ❌'}")
    print(f"기대했던 매칭: {case['expected_match']}")
    print("-" * 80)

## 3. 방법 1: Dense Model Knowledge Distillation ⭐

Dense retrieval 모델(KoSBERT, KoSimCSE)은 동의어를 잘 처리합니다.
이 지식을 Neural Sparse 모델에 전이합니다.

### 3.1. Dense Teacher 모델 로드

In [ ]:
# 한국어 Dense Retrieval 모델 (Teacher)
TEACHER_MODEL = "jhgan/ko-sroberta-multitask"  # 또는 "BM-K/KoSimCSE-roberta"

print(f"Teacher 모델 로드 중: {TEACHER_MODEL}")
teacher_tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL)
teacher_model = AutoModel.from_pretrained(TEACHER_MODEL)
teacher_model = teacher_model.to(device)
teacher_model.eval()

print(f"✓ Teacher 모델 로드 완료")

def encode_dense(text, model, tokenizer, device):
    """Dense 임베딩 생성"""
    encoded = tokenizer(
        text,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # Mean pooling
        embeddings = outputs.last_hidden_state
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        sum_embeddings = torch.sum(embeddings * attention_mask_expanded, 1)
        sum_mask = torch.clamp(attention_mask_expanded.sum(1), min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
    
    return mean_embeddings.cpu().numpy()

### 3.2. Dense 모델로 동의어 쌍 검증

In [ ]:
# 동의어 후보 쌍
synonym_candidates = [
    ("인공지능", "AI"),
    ("인공지능", "artificial intelligence"),
    ("딥러닝", "심층학습"),
    ("딥러닝", "deep learning"),
    ("검색", "탐색"),
    ("검색", "search"),
    ("학습", "훈련"),
    ("학습", "training"),
    ("모델", "model"),
    ("데이터", "자료"),
    ("분석", "해석"),
    ("벡터", "vector"),
    # 반의어 (낮은 유사도 확인용)
    ("검색", "음식"),
    ("인공지능", "날씨"),
]

print("="*80)
print("Dense Model로 동의어 유사도 검증")
print("="*80)

synonym_scores = []

for word1, word2 in synonym_candidates:
    emb1 = encode_dense(word1, teacher_model, teacher_tokenizer, device)
    emb2 = encode_dense(word2, teacher_model, teacher_tokenizer, device)
    
    # Cosine similarity
    similarity = np.dot(emb1[0], emb2[0]) / (np.linalg.norm(emb1[0]) * np.linalg.norm(emb2[0]))
    
    synonym_scores.append({
        'word1': word1,
        'word2': word2,
        'similarity': similarity
    })
    
    is_synonym = "✓" if similarity > 0.7 else "✗"
    print(f"{is_synonym} {word1:15s} <-> {word2:20s}: {similarity:.4f}")

# DataFrame으로 정리
synonym_df = pd.DataFrame(synonym_scores)
synonym_df = synonym_df.sort_values('similarity', ascending=False)

print(f"\n평균 유사도: {synonym_df['similarity'].mean():.4f}")
print(f"동의어 쌍 (>0.7): {len(synonym_df[synonym_df['similarity'] > 0.7])}개")

## 4. 방법 2: Synonym-Aware IDF Enhancement

IDF 딕셔너리에 동의어 정보를 통합합니다.

### 4.1. 한국어 동의어 사전 구축

In [ ]:
# 한국어 동의어 사전 (수동 + 자동 확장)
KOREAN_SYNONYMS = {
    # AI/ML 관련
    "인공지능": ["AI", "artificial intelligence", "에이아이"],
    "딥러닝": ["deep learning", "심층학습", "심층 신경망"],
    "머신러닝": ["machine learning", "기계학습"],
    "학습": ["훈련", "training", "learning"],
    "모델": ["model", "알고리즘"],
    
    # 검색 관련
    "검색": ["search", "탐색", "조회", "찾기"],
    "분석": ["analysis", "해석", "분석"],
    "벡터": ["vector", "embedding", "임베딩"],
    
    # 데이터 관련
    "데이터": ["data", "자료"],
    "정보": ["information", "info"],
    
    # 기술 관련
    "기술": ["technology", "tech", "테크놀로지"],
    "시스템": ["system"],
    "엔진": ["engine"],
}

# 역방향 매핑 (동의어 -> 대표어)
synonym_to_canonical = {}
for canonical, synonyms in KOREAN_SYNONYMS.items():
    synonym_to_canonical[canonical] = canonical
    for syn in synonyms:
        synonym_to_canonical[syn] = canonical

print(f"동의어 사전: {len(KOREAN_SYNONYMS)}개 대표어")
print(f"총 {sum(len(v)+1 for v in KOREAN_SYNONYMS.values())}개 단어 포함")

# 예시 출력
print("\n예시:")
for canonical, synonyms in list(KOREAN_SYNONYMS.items())[:3]:
    print(f"  {canonical}: {', '.join(synonyms)}")

### 4.2. Synonym-Aware IDF 생성

In [ ]:
def create_synonym_aware_idf(original_idf, tokenizer, synonym_dict, method='max'):
    """
    동의어 정보를 반영한 IDF 생성
    
    Args:
        original_idf: 원본 IDF 딕셔너리
        tokenizer: Tokenizer
        synonym_dict: 동의어 사전
        method: 'max', 'mean', 'sum' 중 선택
    
    Returns:
        enhanced_idf: 강화된 IDF 딕셔너리
    """
    enhanced_idf = original_idf.copy()
    
    # 각 동의어 그룹에 대해
    for canonical, synonyms in synonym_dict.items():
        all_words = [canonical] + synonyms
        
        # 각 단어의 토큰들 수집
        all_tokens = []
        for word in all_words:
            tokens = tokenizer.tokenize(word)
            all_tokens.extend(tokens)
        
        # 토큰들의 IDF 값 수집
        idf_values = []
        for token in set(all_tokens):
            if token in original_idf:
                idf_values.append(original_idf[token])
        
        if not idf_values:
            continue
        
        # IDF 값 통합
        if method == 'max':
            shared_idf = max(idf_values)
        elif method == 'mean':
            shared_idf = np.mean(idf_values)
        else:  # sum
            shared_idf = sum(idf_values)
        
        # 모든 동의어 토큰에 적용
        for token in set(all_tokens):
            enhanced_idf[token] = shared_idf
    
    return enhanced_idf

# 강화된 IDF 생성
print("Synonym-Aware IDF 생성 중...\n")

enhanced_idf_max = create_synonym_aware_idf(idf_dict, tokenizer, KOREAN_SYNONYMS, method='max')
enhanced_idf_mean = create_synonym_aware_idf(idf_dict, tokenizer, KOREAN_SYNONYMS, method='mean')

print("✓ 강화된 IDF 생성 완료")

# 변화 확인
print("\n변화 예시 (MAX 방식):")
test_words = ["인공지능", "AI", "딥러닝", "deep"]
for word in test_words:
    tokens = tokenizer.tokenize(word)
    for token in tokens:
        if token in idf_dict:
            original = idf_dict[token]
            enhanced = enhanced_idf_max[token]
            change = "↑" if enhanced > original else "→"
            print(f"  {token:15s}: {original:.4f} {change} {enhanced:.4f}")

### 4.3. Query Expansion with Synonyms

In [ ]:
def expand_query_with_synonyms(query, synonym_dict, max_expansions=2):
    """
    쿼리를 동의어로 확장
    
    Args:
        query: 원본 쿼리
        synonym_dict: 동의어 사전
        max_expansions: 단어당 최대 확장 수
    
    Returns:
        expanded_query: 확장된 쿼리
    """
    words = query.split()
    expanded_words = []
    
    for word in words:
        expanded_words.append(word)
        
        # 동의어 찾기
        if word in synonym_dict:
            synonyms = synonym_dict[word][:max_expansions]
            expanded_words.extend(synonyms)
    
    return " ".join(expanded_words)

# 테스트
test_queries = [
    "인공지능 검색",
    "딥러닝 모델",
    "데이터 분석",
]

print("Query Expansion 예시:\n")
for query in test_queries:
    expanded = expand_query_with_synonyms(query, KOREAN_SYNONYMS)
    print(f"원본: {query}")
    print(f"확장: {expanded}")
    print()

## 5. 방법 3: 토큰 임베딩 기반 동의어 자동 발견

### 5.1. BERT 토큰 임베딩으로 유사 토큰 찾기

In [ ]:
# KoBERT 로드 (토큰 유사도 계산용)
kobert_model_name = "klue/bert-base"
kobert_model = AutoModel.from_pretrained(kobert_model_name)
kobert_model = kobert_model.to(device)
kobert_model.eval()

# Token embedding 추출
token_embeddings = kobert_model.embeddings.word_embeddings.weight.detach().cpu().numpy()

print(f"✓ Token embeddings: {token_embeddings.shape}")

def find_similar_tokens(token, tokenizer, embeddings, top_k=10, threshold=0.7):
    """
    주어진 토큰과 유사한 토큰들 찾기
    """
    # 토큰 ID
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id == tokenizer.unk_token_id:
        return []

    # 해당 토큰의 임베딩
    token_emb = embeddings[token_id]

    # 모든 토큰과의 코사인 유사도 계산
    similarities = np.dot(embeddings, token_emb) / (
        np.linalg.norm(embeddings, axis=1) * np.linalg.norm(token_emb)
    )

    # 상위 k개
    top_indices = np.argsort(similarities)[-top_k-1:-1][::-1]

    similar_tokens = []
    for idx in top_indices:
        if similarities[idx] >= threshold and idx != token_id:
            # 수정: int()로 변환하여 전달
            similar_token = tokenizer.decode([int(idx)])
            similar_tokens.append((similar_token, float(similarities[idx])))

    return similar_tokens

# 테스트
print("\n유사 토큰 찾기 테스트:\n")
test_tokens = ["검색", "학습", "데이터", "AI"]

for token in test_tokens:
    similar = find_similar_tokens(token, tokenizer, token_embeddings, top_k=5, threshold=0.6)
    print(f"{token}의 유사 토큰:")
    for sim_token, score in similar:
        print(f"  - {sim_token:15s} (유사도: {score:.4f})")
    print()

### 5.2. 자동 동의어 사전 구축

In [ ]:
def build_automatic_synonym_dict(important_tokens, tokenizer, embeddings, threshold=0.75):
    """
    중요 토큰들에 대해 자동으로 동의어 사전 구축
    """
    auto_synonyms = {}
    
    for token in tqdm(important_tokens, desc="Building synonym dict"):
        similar = find_similar_tokens(token, tokenizer, embeddings, top_k=5, threshold=threshold)
        if similar:
            auto_synonyms[token] = [t for t, _ in similar]
    
    return auto_synonyms

# IDF 상위 1000개 토큰에 대해 동의어 찾기
sorted_idf = sorted(idf_dict.items(), key=lambda x: x[1], reverse=True)
important_tokens = [token for token, _ in sorted_idf[:1000] 
                   if not token.startswith('##') and len(token) > 1]

print(f"중요 토큰 {len(important_tokens)}개에 대해 동의어 찾는 중...\n")

# 샘플만 테스트 (전체는 시간 오래 걸림)
auto_synonym_dict = build_automatic_synonym_dict(
    important_tokens[:50],  # 샘플 50개
    tokenizer,
    token_embeddings,
    threshold=0.75
)

print(f"\n✓ 자동 동의어 사전 생성 완료: {len(auto_synonym_dict)}개 엔트리")

# 예시 출력
print("\n예시:")
for token, synonyms in list(auto_synonym_dict.items())[:5]:
    print(f"  {token}: {', '.join(synonyms)}")

## 6. 방법 4: Knowledge Distillation Loss 구현

### 6.1. Distillation Loss 정의

In [ ]:
def knowledge_distillation_loss(student_scores, teacher_scores, temperature=2.0, alpha=0.5):
    """
    Knowledge Distillation Loss
    
    Args:
        student_scores: Sparse 모델의 유사도 점수
        teacher_scores: Dense 모델의 유사도 점수
        temperature: Softmax temperature
        alpha: Loss 가중치 (0~1)
    
    Returns:
        distillation_loss
    """
    # Soft targets from teacher
    soft_teacher = F.softmax(teacher_scores / temperature, dim=-1)
    
    # Soft predictions from student
    soft_student = F.log_softmax(student_scores / temperature, dim=-1)
    
    # KL Divergence
    distillation_loss = F.kl_div(
        soft_student,
        soft_teacher,
        reduction='batchmean'
    ) * (temperature ** 2)
    
    return distillation_loss

print("✓ Knowledge Distillation Loss 정의 완료")
print("\n사용 방법:")
print("  total_loss = alpha * distillation_loss + (1-alpha) * original_loss")

### 6.2. 개선된 학습 손실 함수

In [ ]:
def enhanced_training_loss(
    doc_sparse,
    query_sparse, 
    relevance,
    teacher_doc_emb=None,
    teacher_query_emb=None,
    idf_dict=None,
    lambda_l0=1e-3,
    lambda_idf=1e-2,
    lambda_kd=0.3,
    temperature=2.0
):
    """
    동의어를 고려한 개선된 학습 손실
    
    Args:
        doc_sparse: Student sparse document vectors
        query_sparse: Student sparse query vectors
        relevance: Ground truth relevance
        teacher_doc_emb: Teacher dense document embeddings (optional)
        teacher_query_emb: Teacher dense query embeddings (optional)
        idf_dict: IDF dictionary
        lambda_l0: L0 regularization weight
        lambda_idf: IDF-aware penalty weight
        lambda_kd: Knowledge distillation weight
        temperature: KD temperature
    
    Returns:
        total_loss, loss_dict
    """
    # 1. Ranking Loss (Sparse)
    sparse_similarity = torch.sum(doc_sparse * query_sparse, dim=-1)
    ranking_loss = F.binary_cross_entropy_with_logits(
        sparse_similarity, relevance, reduction='mean'
    )
    
    # 2. L0 Regularization
    l0_loss = torch.mean(torch.sum(torch.abs(doc_sparse), dim=-1))
    
    # 3. IDF-aware Penalty
    if idf_dict is not None:
        idf_tensor = torch.tensor(
            [idf_dict.get(i, 1.0) for i in range(doc_sparse.shape[1])],
            device=doc_sparse.device
        )
        inverse_idf = 1.0 / (idf_tensor + 1e-6)
        idf_penalty = torch.mean(torch.sum(doc_sparse * inverse_idf, dim=-1))
    else:
        idf_penalty = torch.tensor(0.0)
    
    # 4. Knowledge Distillation from Dense Teacher (optional)
    kd_loss = torch.tensor(0.0)
    if teacher_doc_emb is not None and teacher_query_emb is not None:
        # Dense similarity
        dense_similarity = torch.sum(
            teacher_doc_emb * teacher_query_emb, dim=-1
        )
        
        # KD Loss
        kd_loss = knowledge_distillation_loss(
            sparse_similarity.unsqueeze(-1),
            dense_similarity.unsqueeze(-1),
            temperature=temperature
        )
    
    # Total Loss
    total_loss = (
        ranking_loss + 
        lambda_l0 * l0_loss + 
        lambda_idf * idf_penalty +
        lambda_kd * kd_loss
    )
    
    loss_dict = {
        'total': total_loss.item(),
        'ranking': ranking_loss.item(),
        'l0': l0_loss.item(),
        'idf_penalty': idf_penalty.item(),
        'kd': kd_loss.item() if isinstance(kd_loss, torch.Tensor) else 0.0
    }
    
    return total_loss, loss_dict

print("✓ 개선된 학습 손실 함수 정의 완료")

## 7. 성능 비교 테스트

### 7.1. 원본 vs Synonym-Enhanced IDF 비교

In [ ]:
def encode_query_with_idf(query, tokenizer, idf_dict):
    """쿼리를 IDF로 인코딩"""
    tokens = tokenizer.encode(query, add_special_tokens=False)
    sparse_vec = np.zeros(tokenizer.vocab_size)
    for token_id in tokens:
        token_str = tokenizer.decode([token_id])
        if token_str in idf_dict:
            sparse_vec[token_id] = idf_dict[token_str]
    return sparse_vec

# 테스트 케이스
test_cases = [
    {
        "query": "인공지능 검색",
        "doc1": "AI 기반 탐색 시스템",  # 동의어 매칭
        "doc2": "날씨 예보 서비스",      # 무관
    },
    {
        "query": "딥러닝 학습",
        "doc1": "심층 신경망 훈련 방법",
        "doc2": "요리 레시피 모음",
    },
]

print("="*80)
print("원본 IDF vs Synonym-Enhanced IDF 성능 비교")
print("="*80)

for case in test_cases:
    query = case['query']
    doc1 = case['doc1']
    doc2 = case['doc2']
    
    print(f"\n쿼리: {query}")
    print(f"  관련 문서: {doc1}")
    print(f"  무관 문서: {doc2}")
    
    # 원본 IDF
    q_vec_orig = encode_query_with_idf(query, tokenizer, idf_dict)
    d1_vec_orig = encode_query_with_idf(doc1, tokenizer, idf_dict)
    d2_vec_orig = encode_query_with_idf(doc2, tokenizer, idf_dict)
    
    sim1_orig = np.dot(q_vec_orig, d1_vec_orig)
    sim2_orig = np.dot(q_vec_orig, d2_vec_orig)
    
    # Enhanced IDF
    q_vec_enh = encode_query_with_idf(query, tokenizer, enhanced_idf_max)
    d1_vec_enh = encode_query_with_idf(doc1, tokenizer, enhanced_idf_max)
    d2_vec_enh = encode_query_with_idf(doc2, tokenizer, enhanced_idf_max)
    
    sim1_enh = np.dot(q_vec_enh, d1_vec_enh)
    sim2_enh = np.dot(q_vec_enh, d2_vec_enh)
    
    print(f"\n  [원본 IDF]")
    print(f"    관련 문서 유사도: {sim1_orig:.4f}")
    print(f"    무관 문서 유사도: {sim2_orig:.4f}")
    print(f"    차이: {sim1_orig - sim2_orig:.4f}")
    
    print(f"\n  [Enhanced IDF]")
    print(f"    관련 문서 유사도: {sim1_enh:.4f} {'↑' if sim1_enh > sim1_orig else ''}")
    print(f"    무관 문서 유사도: {sim2_enh:.4f}")
    print(f"    차이: {sim1_enh - sim2_enh:.4f} {'✓ 개선' if (sim1_enh - sim2_enh) > (sim1_orig - sim2_orig) else ''}")
    
    print("-" * 80)

## 8. 권장 사항 및 다음 단계

In [ ]:
recommendations = """
╔═══════════════════════════════════════════════════════════════════════════╗
║                    Neural Sparse 성능 개선 권장 사항                        ║
╚═══════════════════════════════════════════════════════════════════════════╝

## 🎯 우선순위별 개선 방법

### 1순위: Knowledge Distillation ⭐⭐⭐
   - Dense 모델(KoSBERT)로부터 knowledge distillation
   - 효과: 동의어 처리 능력 크게 향상
   - 구현: enhanced_training_loss() 사용
   - 예상 성능 향상: 15-20%

### 2순위: Synonym-Aware IDF ⭐⭐
   - 수동/자동 동의어 사전으로 IDF 강화
   - 효과: 즉시 적용 가능, 재학습 불필요
   - 구현: enhanced_idf_max 사용
   - 예상 성능 향상: 5-10%

### 3순위: Query Expansion ⭐
   - 검색 시점에 쿼리 확장
   - 효과: 검색 recall 향상
   - 구현: expand_query_with_synonyms() 사용
   - 예상 성능 향상: 3-7%

## 📝 구체적 실행 계획

### 단계 1: Synonym-Enhanced IDF 적용 (즉시 가능)
```python
# 1. Enhanced IDF 저장
with open('idf_enhanced.json', 'w') as f:
    json.dump(enhanced_idf_max, f)

# 2. Inference 시 사용
model = NeuralSparseSearchModel.from_pretrained(
    model_dir,
    idf_dict=enhanced_idf_max  # Enhanced IDF 사용
)
```

### 단계 2: Knowledge Distillation으로 재학습 (1-2일)
```python
# Teacher 모델 준비
teacher_model = AutoModel.from_pretrained("jhgan/ko-sroberta-multitask")

# 학습 시 KD loss 추가
loss, loss_dict = enhanced_training_loss(
    doc_sparse, query_sparse, relevance,
    teacher_doc_emb, teacher_query_emb,  # Teacher embeddings
    lambda_kd=0.3  # KD weight
)
```

### 단계 3: 동의어 사전 확장 (지속적)
- 자동 동의어 발견 시스템 구축
- 도메인별 동의어 추가
- 사용자 피드백 반영

## 🔬 추가 연구 방향

1. **Multi-Vector Sparse Encoding**
   - 여러 sparse vectors로 문서 표현
   - 다양한 의미 측면 포착

2. **Cross-Lingual Transfer**
   - 영어 모델의 지식을 한국어로 전이
   - 외래어 처리 개선

3. **Dynamic IDF**
   - 검색 로그 기반 IDF 업데이트
   - 실시간 트렌드 반영

4. **Hybrid Retrieval**
   - Sparse + Dense 앙상블
   - 각각의 장점 활용

## 📊 예상 최종 성능

현재 성능:     ████░░░░░░ (40%)
Enhanced IDF:  ██████░░░░ (55%)
KD 재학습:      ████████░░ (75%)
전체 적용:      █████████░ (85%)

## ⚠️  주의사항

1. **Trade-off 고려**
   - Query expansion은 속도 저하 가능
   - KD 학습은 시간 소요

2. **평가 필수**
   - 실제 검색 로그로 평가
   - MRR, NDCG, Recall@K 측정

3. **점진적 적용**
   - A/B 테스트로 검증
   - 단계별로 성능 확인
"""

print(recommendations)

## 9. 개선된 IDF 저장

In [ ]:
# Enhanced IDF 저장
output_dir = "./opensearch-korean-neural-sparse-v1-enhanced"
os.makedirs(output_dir, exist_ok=True)

# 1. Enhanced IDF (MAX 방식)
with open(os.path.join(output_dir, "idf_enhanced_max.json"), 'w', encoding='utf-8') as f:
    json.dump(enhanced_idf_max, f, ensure_ascii=False, indent=2)

# 2. Enhanced IDF (MEAN 방식)
with open(os.path.join(output_dir, "idf_enhanced_mean.json"), 'w', encoding='utf-8') as f:
    json.dump(enhanced_idf_mean, f, ensure_ascii=False, indent=2)

# 3. 동의어 사전
with open(os.path.join(output_dir, "synonym_dict.json"), 'w', encoding='utf-8') as f:
    json.dump(KOREAN_SYNONYMS, f, ensure_ascii=False, indent=2)

# 4. 자동 동의어 사전
with open(os.path.join(output_dir, "auto_synonym_dict.json"), 'w', encoding='utf-8') as f:
    json.dump(auto_synonym_dict, f, ensure_ascii=False, indent=2)

print(f"✓ 개선된 IDF 및 동의어 사전 저장 완료")
print(f"  저장 위치: {output_dir}/")
print(f"\n저장된 파일:")
for filename in os.listdir(output_dir):
    filepath = os.path.join(output_dir, filename)
    size = os.path.getsize(filepath) / 1024
    print(f"  - {filename:30s} ({size:>8.2f} KB)")

## 10. 다음 단계: Re-training Script 생성

In [ ]:
retraining_guide = """
╔═══════════════════════════════════════════════════════════════════════════╗
║                Knowledge Distillation 재학습 가이드                        ║
╚═══════════════════════════════════════════════════════════════════════════╝

## 재학습을 위한 수정 사항

### 1. korean_neural_sparse_training.ipynb 수정

#### 변경 1: Teacher 모델 추가
```python
# 셀 추가: Teacher 모델 로드
teacher_model = AutoModel.from_pretrained("jhgan/ko-sroberta-multitask")
teacher_model = teacher_model.to(device)
teacher_model.eval()
```

#### 변경 2: 학습 루프 수정
```python
def train_epoch_with_kd(...):
    for batch in progress_bar:
        # ... 기존 코드 ...
        
        # Teacher embeddings 생성
        with torch.no_grad():
            teacher_doc_emb = teacher_model(
                doc_input_ids, doc_attention_mask
            ).last_hidden_state.mean(dim=1)
            
            teacher_query_emb = teacher_model(
                query_input_ids, query_attention_mask  
            ).last_hidden_state.mean(dim=1)
        
        # Enhanced loss 사용
        loss, loss_dict = enhanced_training_loss(
            doc_sparse, query_sparse, relevance,
            teacher_doc_emb, teacher_query_emb,
            idf_id_dict,
            lambda_l0=1e-3,
            lambda_idf=1e-2,
            lambda_kd=0.3  # KD weight (중요!)
        )
```

#### 변경 3: Enhanced IDF 사용
```python
# IDF 계산 후 동의어로 강화
idf_token_dict_enhanced = create_synonym_aware_idf(
    idf_token_dict,
    tokenizer,
    KOREAN_SYNONYMS,
    method='max'
)
```

### 2. 하이퍼파라미터 조정

```python
# 추천 설정
LEARNING_RATE = 1e-5      # KD 시 낮게 설정
NUM_EPOCHS = 5            # 더 많은 epoch
BATCH_SIZE = 16           # 메모리 허용 시 증가
LAMBDA_KD = 0.3           # KD loss 가중치
TEMPERATURE = 2.0         # Distillation temperature
```

### 3. 평가 추가

```python
# 동의어 매칭 성능 평가
synonym_test_cases = [
    ("인공지능", "AI"),
    ("딥러닝", "심층학습"),
    ("검색", "탐색"),
]

for word1, word2 in synonym_test_cases:
    vec1 = encode_query_inference_free(word1, tokenizer, idf_dict)
    vec2 = encode_query_inference_free(word2, tokenizer, idf_dict)
    similarity = np.dot(vec1, vec2)
    print(f"{word1} <-> {word2}: {similarity:.4f}")
```

## 예상 학습 시간

- GPU: Tesla T4 기준
- 데이터: 30,000 pairs
- Epoch: 5회
- 예상 시간: 약 2-3시간

## 체크리스트

- [ ] Teacher 모델 로드 확인
- [ ] Enhanced IDF 생성
- [ ] KD loss 구현
- [ ] 하이퍼파라미터 조정
- [ ] 동의어 테스트 케이스 준비
- [ ] 학습 전 베이스라인 성능 측정
- [ ] 학습 후 성능 비교
"""

print(retraining_guide)